![Astrofisica Computacional](../logo.PNG)

---
## 01. Barnes-Hut Algorithm

Eduard Larrañaga (ealarranaga@unal.edu.co)

---

### Summary

The Barnes-Hut method to describe the dynamics of an N-body gravitational system is presented.


---

## The Barnes-Hut Algorithm

The Barnes-Hut algorithm (initially described by Josh Barnes and Piet Hut) presents a solution method for the N-body gravitational problem that manages to reduce the order $\mathcal{O} (N^2)$ of the direct addition methods (brute force) to order $\mathcal{O} (N \log N)$.

The central idea of ​​this algorithm corresponds to dividing the physical space where the particles are located in cubic cells to form an **octo-tree** (if the space is 3D) or in square cells to form a **square-tree* * (if the space is 2D). A quad-tree (*quad-tree*) is similar to a binary tree but each node has 4 *children* (some of which may be empty). In this model, each node represents a region of space.

If there is a set of distant and close together particles, these can be grouped in the form of a single object with the mass of the set and located in a cell located at the center of mass of the group. This description corresponds to describing the set of particles by a low-order multipole expansion (monopole order). By making this approximation, the number of interaction calculations is dramatically reduced.

---
### The Algorithm in 2-Dimensions

Consider a set of N-bodies moving in 2-dimensional space under their mutual gravitational interaction. In the Barnes-Hut description, the set of N-bodies is recursively divided into groups, storing their information in a quad-tree formed by the square cells (quadrants) into which the physical space is divided. In this way, each **node** of the tree corresponds to a region of space.

The initial node (root) represents the entire physical space and four nodes will emerge from it, representing the four quadrants into which that space is divided. Then each node of this second order continues recursively dividing into quadrants until such time as each of them contains a single particle or this void. In the following figure you can see the generated tree for a 2-dimensional space with 8 particles:

![](BarnesHutTree.001.jpeg)

Thus, there are three types of nodes in the tree:

- External nodes: It does not have *children* nodes, that is, it is not sub-divided. At these nodes there is a particle or it is empty. In case of including a particle, the node must store the current information of the mass, position and speed of the particle.

- Internal nodes: Each internal node has sub-divisions and represents the entire group of particles that are located in the child nodes it has. The internal node stores the information of the total mass of the particles below it along with the position of the corresponding center of mass.

- Empty Nodes: They represent an empty quadrant.

The tree that describes the physical system must be updated at each step of the integration process.

**Tree Construction**

For the construction of the tree, the bodies are inserted one by one following this recursion procedure:

1. If the node you want to put on does not contain bodies, the body is placed there and the value of the mass and center of mass associated with the node is updated.
2. If the node is an *internal node* (i.e. it already has child nodes) the value of the total mass and the center of mass are updated, and then proceed to insert the body recursively in the appropriate quadrant.
3. If the node is an *external node* (i.e. it already contains a single particle), then the two bodies will be in the same region. You have to sub-split the node into four children and then insert the two particles into the appropriate quadrant recursively. In the event that the two particles are again located in the same sub-quadrant, it is necessary to sub-divide that quadrant again and continue with the recursion. Finally, the total mass and center of mass must be updated.

#### The Calculation of the Gravitational Force

To calculate the net force on a specific particle $N_k$ of the system at a given instant of time, it is necessary to go through all the nodes of the tree, starting from the *root*. If the center of mass of one of the internal nodes is far enough from the body $N_k$, all the bodies contained by the node will be treated as a single particle whose mass will be the total mass of the set and its location will be given by the position of the respective center of mass. On the other hand, if the internal node is close enough to $N_k$, it is necessary to repeat the process with each of its child nodes.

The closeness criterion used is made by comparing the ratio $\frac{s}{d}$ (with $s$ the side of the quadrant described by the internal node and $d$ the distance from $N_k$ to the center of mass stored at the internal node) with a cutoff value $\theta$ that defines the precision of the algorithm. In this way, if $\frac{s}{d} <\theta$ the node is said to be sufficiently far from $N_k$, while if $\frac{s}{d} >\theta$, the node is too close and its child nodes must be considered.

**Note**: If $\theta =0$ is taken, no internal nodes are considered as a single body and the Barnes-Hut algorithm becomes the direct-sum (brute force) algorithm considered in previous classes.

The process to compute the net force on a body **b** starts from the root node and continues with the following recursion over all nodes:

1. If the current node is an *external node* and does not correspond to the body **b** (to avoid self-gravity), the force exerted by the node on **b** is calculated and added to the force net.

2. If the current node is NOT an *external node* the ratio $\frac{s}{d}$ is calculated.
  - If $\frac{s}{d}<\theta$, the node can be treated as a single body. The force exerted on the body **b** is calculated and this value is added to the net force.
 
  - If $\frac{s}{d}>\theta$, it is necessary to proceed cursively on each of the children of the current node.